In [ ]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

from lsst.ts.observatory.control.utils import RotType

In [ ]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
#get classes and start them
domain = salobj.Domain()

In [ ]:
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
tmp = atcs.rem.atmcs.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
tmp = await latiss.rem.atspectrograph.evt_heartbeat.next(flush=True, timeout=5)
print(tmp)

In [ ]:
tmp =latiss.rem.atspectrograph.evt_summaryState.get()
print(tmp)

In [ ]:
await latiss.enable()

In [ ]:
await atcs.enable()

In [ ]:
tmp = latiss.rem.atarchiver.evt_heartbeat.get()
print(tmp)

In [ ]:
tmp = latiss.rem.atarchiver.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
# enable components
await atcs.enable()
#await latiss.enable()

In [ ]:
# ATDOME only
tmp = await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply='test')
tmp = await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
#tmp=await atcs.rem.atdome.cmd_start.set_start(settingsToApply='test')
#tmp=await atcs.rem.atdome.cmd_enable.start()
#print(tmp)

In [ ]:
# ATMCS
tmp = await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)
#tmp=await atcs.rem.atmcs.cmd_start.start(timeout=5)

In [ ]:
# ATHexapod
tmp = await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.ENABLED, settingsToApply='summit')
#tmp = await atcs.rem.athexapod.cmd_start.set_start(settingsToApply='auxtel_201912.yaml')

In [ ]:
# ATPneumatics
tmp = await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.ENABLED)
#tmp=await atcs.rem.atpneumatics.cmd_start.start()

In [ ]:
# ATPtg
tmp = await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)
#tmp=await atcs.rem.atptg.cmd_enable.start()
# Need to manually say which rotator we're using at the moment (N2-->focus=3, N1-->focus=2)
#await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [ ]:
# ATAOS
tmp = await salobj.set_summary_state(atcs.rem.ataos, salobj.State.ENABLED)
#tmp=await atcs.rem.ataos.cmd_enable.start()

In [ ]:
await atcs.rem.atdome.cmd_homeAzimuth.start()

In [ ]:
# This doesn't move anything! 
tmp=await atcs.rem.atdome.cmd_moveAzimuth.set_start(5)
print(tmp)

In [ ]:
# This does
tmp=await atcs.rem.atdome.cmd_moveAzimuth.set_start(azimuth=0)
print(tmp)

In [ ]:
# ATHexapod
tmp = await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY, settingsToApply='summit')
#tmp=await atcs.rem.athexapod.cmd_start.set_start(settingsToApply='summit')
#tmp=await atcs.rem.athexapod.cmd_enable.start()
print(tmp)

In [ ]:
# ATSPECTROGRAPH
tmp = await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY, settingsToApply='LATISS_AuxTel_20210318_v3.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_start.set_start(settingsToApply='LATISS_AuxTel_20210318_v3.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_enable.start()
print(tmp)

In [ ]:
await atcs.rem.atmcs.

In [ ]:
await atcs.shutdown()

In [ ]:
tmp=await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.start()
print(tmp)

In [ ]:
tmp=await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.start()
print(tmp)

In [ ]:
tmp = await atcs.open_valves()
print(tmp)

In [ ]:
tmp = await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.start()

In [ ]:
tmp = await atcs.rem.atpneumatics.cmd_closeM1CellVents.start()
print(tmp)

In [ ]:
tmp = await atcs.rem.atpneumatics.cmd_closeM1Cover.start()
print(tmp)

In [ ]:
tmp = await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.start()

In [ ]:
# take event checking out the slew commands to test telescope only
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
offset = 1.12 #2./3600
start_az=0+offset
start_el=80+offset
start_rot_pa=1.5+offset
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
# Perform Rotator verification (LTS-337-003)True

In [ ]:
await atcs.rem.atmcs.cmd_abort.start()

In [ ]:
await atcs.point_azel(az=10, el=40)

In [ ]:
await atcs.stop_tracking()

In [ ]:
tmp = await atcs.rem.atmcs.cmd_stopTracking.start(timeout=15)
print(tmp)

In [ ]:
start_az=-58
start_el=35
start_rot_pa=13.1
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
start_az=165.5
start_el=67.9
start_rot_pa=170

start_time =time.time()
await atcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)
end_time = time.time()
print(f'Time to perform {170*2} degree rotation of N2 is {end_time-start_time} seconds')

In [ ]:
# send telescope to alt/az/
#await salobj.set_summary_state(latiss.atcamera, salobj.State.ENABLED)

In [ ]:
# point telescope to desired starting position
# this make take extra time as more checks are performed
start_az=0
start_el=70
start_rot_pa=0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
#declare offset size
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 10 # degrees

d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

In [ ]:
await atcs.stop_tracking()

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

# Perform tracking test by taking continous 20s exposures
#await attcs.slew_object('HD 167060')
#await atcs.slew_object('HIP 100787')
await atcs.slew_object('51 Cap')

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")
await atcs.stop_tracking()

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")
await atcs.stop_tracking()

In [ ]:
await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')

In [ ]:
await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
# stopped tracking elsewhere, then reslewed
await atcs.slew_object('HR 8496')

In [ ]:
await atcs.slew_object('HR 280')

In [ ]:
# turn on corrections
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True, timeout=10)

In [ ]:
tmp = await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)

In [ ]:
# is ATAOS correcting?
tmp = await atcs.rem.ataos.evt_correctionEnabled.aget()
print(tmp)

In [ ]:
tmp = await atcs.rem.ataos.evt_
print(tmp)

In [ ]:
tmp = await atcs.rem.atpneumatics.tel_m1AirPressure.aget()
print(tmp)

In [ ]:
# get offsets from ATAOS - after turning on atspectrograph correction
# this had a filter in the beam? why is it not showing?  -- I think the display was just stale
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# Verify that ATAOS handles focus/pointing offsets from grating change
await latiss.rem.atspectrograph.cmd_changeFilter.set_start(filter=0)
await latiss.rem.atspectrograph.cmd_changeDisperser.set_start(disperser=0)

In [ ]:
# get offsets from ATAOS - after turning on atspectrograph correction
# this had a filter in the beam? why is it not showing?
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# Verify that ATAOS handles focus/pointing offsets from grating change
await latiss.rem.atspectrograph.cmd_changeFilter.set_start(filter=0)
#await latiss.rem.atspectrograph.cmd_changeDisperser.set_start(disperser=0)

In [ ]:
# get offsets from ATAOS - after turning on atspectrograph correction
# this had a filter in the beam? why is it not showing?
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = await atcs.rem.ataos.evt_pointingOffsetSummary.aget()
print(pointing_offsets)

In [ ]:
# now apply user offset
await atcs.rem.ataos.cmd_offset.set_start(z=1.34)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = atcs.rem.ataos.evt_pointingOffsetSummary.get()
print(pointing_offsets)

In [ ]:
# reset offsets
tmp = await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
await latiss.take_darks(60, 1)

In [ ]:
# take LATISS images
await latiss.take_bias(nbias=10)

In [ ]:
# get offsets from ATAOS
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
atcs.rem.ataos.evt_focusOffsetSummary.flush()
atcs.rem.ataos.evt_pointingOffsetSummary.flush()
atcs.rem.ataos.evt_atspectrographCorrectionStarted.flush()
atcs.rem.ataos.evt_atspectrographCorrectionCompleted.flush()
await latiss.rem.atspectrograph.cmd_changeFilter.set_start(filter=1)
await atcs.rem.ataos.evt_atspectrographCorrectionStarted.next(flush=False,timeout=10)
await atcs.rem.ataos.evt_atspectrographCorrectionCompleted.next(flush=False,timeout=10)
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.next(flush=False,timeout=10)
pointing_offsets = await atcs.rem.ataos.evt_pointingOffsetSummary.next(flush=False,timeout=10)
print(focus_offsets)
print(pointing_offsets)

In [ ]:
await latiss.rem.atspectrograph.cmd_changeDisperser.set_start(disperser=0)

In [ ]:
atcs.rem.ataos.evt_focusOffsetSummary.flush()
atcs.rem.ataos.evt_pointingOffsetSummary.flush()
atcs.rem.ataos.evt_atspectrographCorrectionStarted.flush()
atcs.rem.ataos.evt_atspectrographCorrectionCompleted.flush()
await latiss.setup_atspec(filter=0, grating=0)
await atcs.rem.ataos.evt_atspectrographCorrectionStarted.next(flush=False,timeout=10)
await atcs.rem.ataos.evt_atspectrographCorrectionCompleted.next(flush=False,timeout=10)
await atcs.rem.ataos.evt_atspectrographCorrectionStarted.next(flush=False,timeout=10)
await atcs.rem.ataos.evt_atspectrographCorrectionCompleted.next(flush=False,timeout=10)
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.next(flush=False,timeout=10)
pointing_offsets = await atcs.rem.ataos.evt_pointingOffsetSummary.next(flush=False,timeout=10)
print(focus_offsets)
print(pointing_offsets)

In [ ]:
def print_allAxesInPosition_log_message(id_data):
    print(f'ATMCS allAxesInPosition event: {id_data}')
# def print_inPosition_log_message(id_data):
#     print(f'Hexapod inPosition event: {id_data}')

In [ ]:
atcs.rem.atmcs.evt_allAxesInPosition.callback = None

In [ ]:
atmcs_rem1.evt_allAxesInPosition.callback = print_allAxesInPosition_log_message
# atcs.rem.athexapod.evt_inPosition.callback = print_inPosition_log_message

In [ ]:
asyncio.get_event_loop().run_forever()

In [ ]:
await atcs.rem.ataos.cmd_offset.set_start(z=-0.1)

In [ ]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
await atcs.prepare_for_flatfield()

In [ ]:
await atcs.rem.atpneumatics.cmd_closeM1Cover.start()

In [ ]:
await atcs.point_azel(atcs.tel_park_az, atcs.tel_park_el, atcs.tel_park_rot, wait_dome=False)

In [ ]:
await atcs.stop_tracking()

In [ ]:
domain = salobj.Domain()
atmcs_rem1 = salobj.Remote(name="ATMCS", domain=domain)

In [ ]:
await atmcs_rem1.start_task

In [ ]:
atmcs_rem1.evt_allAxesInPosition.callback = print_allAxesInPosition_log_message

In [ ]:
offset= 1
atcs.check.atdometrajectory = False
atcs.check.atdome = False
await atcs.point_azel(atcs.tel_park_az+offset, atcs.tel_park_el+offset, atcs.tel_park_rot, wait_dome=False)

In [ ]:
atcs.tel_park_az

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.slew_object('HD 10958', rot_type=RotType.PhysicalSky, rot=-91)

In [ ]:
await atcs.prepare_for_flatfield()

In [ ]:
atcs.check.atdome=True

In [ ]:
await atcs.shutdown()

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.DISABLED)

In [ ]:
atcs.check.atdometrajectory = False

In [ ]:
await atcs.point_azel(target_name="Park position", az=atcs.tel_park_az, el=atcs.tel_park_el, rot_tel=atcs.tel_park_rot, wait_dome=False)

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.shutdown()

In [ ]:
# ht = PointD(2036.5, 2000.5)
x=943-2036.5
pixel_scale = 0.09569

In [ ]:
y=2934 - 2000.5

In [ ]:
print(x)
print(y)

In [ ]:
x*pixel_scale

In [ ]:
y*pixel_scale

In [ ]:
from astropy.time import Time, TimeDelta
print(Time(Time.now(), format='isot', scale='tai'))
await latiss.take_engtest(2)

In [ ]:
await latiss.take_bias(nbias=1)

In [ ]:
print(Time(Time.now(), format='isot', scale='tai'))
tmp=await latiss.rem.atcamera.cmd_abort.start()
print(tmp)

In [ ]:
# turn on corrections
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=False, atspectrograph=False)

In [ ]:
await atcs.point_azel(az=0, el=50)

In [ ]:
m1_fit_z=np.array((114351.58620133, -220485.72530885,  278187.27268067,
        -48810.10245923))
poly_z = np.poly1d(m1_fit_z)

In [ ]:
el_demand=20
pressure_fit=poly_z(np.cos(np.radians(90. - el_demand)))
pressure_psi = 0.000145038 * pressure_fit
print(f'Pressure command is {pressure_fit:0.1f} Pa, which is {pressure_psi:0.4f} PSI')

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=pressure_fit)

In [ ]:
#so set the pressure we want, then move there
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=pressure_fit)
await asyncio.sleep(1)
await atcs.point_azel(az=0, el=el_demand)

In [ ]:
tmp = await atcs.rem.ataos.evt_settingsApplied.aget()
print(tmp)

In [ ]:
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=False, atspectrograph=False)